# model_train notebook

The model_training notebook uses the 7th dataset file of the IEEE FLAME dataset to traing a classification model able to identify aerial views of fire and no fire.

https://ieee-dataport.org/open-access/flame-dataset-aerial-imagery-pile-burn-detection-using-drones-uavs




In [1]:
# Import libraries

import os.path
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Define model parameters

# Add random transformations to training images to slow down overfitting due to
# smaller dataset
data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
        ]
    )

new_size = {'width': 256, 'height': 256}
Config_classification = {'batch_size': 32, 'Save_Model': True, 'Epochs': 5,
                         'TrainingPlot': True}

# Define size of images, batch size, and # of epochs
image_size = (new_size.get('width'), new_size.get('height'))
batch_size = Config_classification.get('batch_size')
save_model_flag = Config_classification.get('Save_Model')
epochs = Config_classification.get('Epochs')

# Create metrics for visuals
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.Accuracy(name='accuracy'),
    keras.metrics.BinaryAccuracy(name='bin_accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc')
]

Train a DNN model based on Keras and Tensorflow as a backend. At first, the directory of Fire and Non_Fire images should be defined for the model, then the model is defined, compiled and fitted over the training and validation set. At the end, the models is saved based on the *.h5 parameters and weights. Training accuracy and loss are demonstrated at the end of this function.

https://keras.io/examples/vision/image_classification_from_scratch/

In [4]:
# Load dataset images
dir_fire = 'gdrive/MyDrive/Colab Notebooks/Frames/Training/Fire/'
dir_no_fire = 'gdrive/MyDrive/Colab Notebooks/Frames/Training/No_Fire/'

In [6]:
# Create training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "gdrive/MyDrive/Colab Notebooks/Frames/Training", validation_split=0.2, subset="training", seed=1337, image_size=image_size,
    batch_size=batch_size, shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "gdrive/MyDrive/Colab Notebooks/Frames/Training", validation_split=0.2, subset="validation", seed=1337, image_size=image_size,
    batch_size=batch_size, shuffle=True
)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

Found 39624 files belonging to 2 classes.
Using 31700 files for training.
Found 39624 files belonging to 2 classes.
Using 7924 files for validation.


In [ ]:
# Function to make Keras model
def make_model_keras(input_shape, num_classes):
    """
    This function define the DNN Model based on the Keras example.
    :param input_shape: The requested size of the image
    :param num_classes: In this classification problem, there are two classes: 1) Fire and 2) Non_Fire.
    :return: The built model is returned
    """
    inputs = keras.Input(shape=input_shape)
    # x = data_augmentation(inputs)  # 1) First option
    x = inputs  # 2) Second option

    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    # x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.Conv2D(8, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x

    # for size in [128, 256, 512, 728]:
    for size in [8]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(size, 1, strides=2, padding="same")(previous_block_activation)

        x = layers.add([x, residual])
        previous_block_activation = x
    x = layers.SeparableConv2D(8, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs, name="model_fire")

In [ ]:
# Create model
model = make_model_keras(input_shape=image_size + (3,), num_classes=2)
model.save('untrained_model.h5')

In [7]:
model = tf.keras.models.load_model('untrained_model.h5')

In [ ]:
# Compile and train model
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"])

res_fire = model.fit(train_ds, epochs=epochs, validation_data=val_ds, batch_size=batch_size)

Epoch 1/5
269/991 [=======>......................] - ETA: 1:16:28 - loss: 0.3927 - accuracy: 0.8296